# [Spotahome.com](https://www.spotahome.com/s/london--uk/for-rent:apartments/for-rent:studios/bedrooms:1/bedrooms:2/bedrooms:3/bedrooms:3more?areaId[]=219&areaId[]=231&areaId[]=232&areaId[]=233&areaId[]=234&areaId[]=235&areaId[]=236&areaId[]=237&areaId[]=241)

## Dataframe with information from search result page

In [1]:
import requests                 # requests on websites
from bs4 import BeautifulSoup   # html parsing

import pandas as pd             # pandas for data frame
import numpy as np              # numpy for numerical operations

import math                     # for math methods
import time                     # for sleep timer
import random                   # for random number generator

import sql_functions as sf      # for sql functions
from sql_functions import *     # functions from file for upload on schema

import sqlalchemy               # for sql connection
import psycopg2                 # for upload on engine

import datetime as dt           # for the csv file with the current date and time
from datetime import date       # for the csv file with the current date and time
import re                       # for regular expressions

import matplotlib as mpl        # for plotting
import matplotlib.pyplot as plt # for plotting
import seaborn as sns           # for plotting

-----

In [2]:
# Creating a function to get all the descriptions
def get_description(bs):
    # find all the descriptions and save them to an empty list
    lst_name = []
    descriptions = bs.find_all(
        class_='homecard-content__title__HomecardContent___OmV4c homecard-content__title--rebranding-style__HomecardContent___OmV4c')
    # iterate over the descriptions to get the text and strip the strings and save them in a list
    for description in descriptions:
        lst_name.append(
            description.get_text()
                .strip()
        )
    return lst_name


# Creating a function to get all the housing types
def get_housing(bs):
    # find all the housing types and save them to an empty list
    lst_name = []
    housings = bs.find_all(
        class_='homecard-content__type__HomecardContent___OmV4c homecard-content__type--rebranding-style__HomecardContent___OmV4c')
    # iterate over the housing types to get the text and strip the strings and save them in a list
    for housing in housings:
        lst_name.append(
            housing.get_text()
                .strip()
        )
    return lst_name


# Creating a function to get all the available dates
def get_available(bs):
    # find all the available dates and save them to an empty list
    lst_name = []
    availables = bs.find_all(
        class_='homecard-content__available-from__HomecardContent___OmV4c homecard-content__available-from--rebranding-style__HomecardContent___OmV4c')
    # iterate over the available appartements to get the text and strip the string and save them in a list
    for available in availables:
        lst_name.append(
            available.get_text()
                .strip()
                .replace('From ', '')
        )
    return lst_name


# Creating a function to get all the prices
def get_price(bs):
    # find all the prices and save them to an empty list
    lst_name = []
    prices = bs.find_all(class_='price__Price___OmV4c')
    # iterate over the prices to get the text and strip the strings and save them in a list
    for price in prices:
        lst_name.append(
            price.get_text()
                .strip()
                .replace('£', '')

        )
    return lst_name


# # Creating a function to get all the prices per period
# def get_prices_period(bs):
#     # find all the aprices per period and save them to an empty list
#     lst_name = []
#     prices_period = bs.find_all(
#         class_='price-monthly__Price___OmV4c price-monthly--rebranding-style__Price___OmV4c')
#     # iterate over the prices per period to get the text and strip the string and save them in a list
#     for price_period in prices_period:
#         lst_name.append(
#             price_period.get_text()
#                 .strip()
#                 .replace('/', '')
#         )
#     return lst_name


# Creating a function to get all the ID's
def get_ids(bs):
    # find all the prices and save them to an empty list
    lst_name = []
    ids = bs.find_all(class_='l-list__item')
    # iterate over the prices to get the text and strip the strings and save them in a list
    for id in ids:
        lst_name.append(
            id.get('data-homecard-scroll')
                .strip()
        )
    return lst_name


# Create dictionary in which every location ID gets assigned a location name
location_dict = {219: 'Lambeth',
                 231: 'Hammersmith and Fulham',
                 232: 'Kensington and Chelsea',
                 233: 'City of Westminster',
                 234: 'Camden',
                 235: 'Tower Hamlets',
                 236: 'Islington',
                 237: 'Hackney',
                 241: 'City of London'
                 }


# Creating a function to get the search result from all pages
# the website spotahome shows 60 search results per page. To iterate trough all the pages, we get the information how many search results are there, then divide it by 60 and round it up to get the number of pages.
def page_results(property_type, location):
    # get the url from the website with the property type and the location as a variable to iterate trough it
    page = requests.get(
        f'https://www.spotahome.com/s/london--uk/for-rent:{property_type}?areaId[]={location}')
    html = page.content
    bs = BeautifulSoup(html, 'html.parser')

    # Extracting the total number of search results
    results = bs.find_all('h1', {'class': 'search-title__title'})

    # define the variable result_text in case the first search gives us no results
    result_text = 0

    # iterate over the results to get the text and strip the string
    for result in results:
        result_text = result.find("strong").get_text().strip()

    # convert the extracted string to an integer to perform mathematical operations
    result_converted = int(result_text)

    # divide the converted result by 60 since one pages shows 60 results and round it up to get the number of pages
    page_site = result_converted / 60
    page_site = math.ceil(page_site)

    # convert the number of pages from a float to an integer to iterate trough the pages
    page_converted = int(page_site)

    # create an empty data frame to store the results from every loop cycle
    df_search = pd.DataFrame()

    # split the url to get access to the part where the page is definde
    begin = f'https://www.spotahome.com/s/london--uk/for-rent:{property_type}'
    end = f'?areaId[]={location}'

    # range is including in the beginning and excluding in the end so we add plus 1 to iterate through all calculated pages
    page_converted = page_converted + 1

    # for loop to get the page numbers
    for page_number in range(page_converted):
        # sleep timer to reduce the traffic for the server
        time.sleep(random.randint(2, 6)/10)

        # get the url from the website with the property type, the location and the page number as a variable to iterate trough it
        page = requests.get(begin+f'/page:{page_number}'+end)
        html = page.content
        bs = BeautifulSoup(html, 'html.parser')

        # Create a dictionary to store the results from every loop cycle.
        # The keys are the column names and the values are the functions we created before.
        # The functions are called with the beautiful soup object as a parameter.
        spotahome_dict = {
            'platform_id': get_ids(bs),
            'platform': 'Spotahome',
            'neighbourhood': location_dict[location],
            'furniture': 'furnished',
            'property_type': property_type,
            'housing_type': get_housing(bs),
            'price': get_price(bs),
            'available_from': get_available(bs),
            'available_today': '',
            'let_type': '',
            'detailed_furniture': 'furnished',
            'scraping_date': date.today(),
            'title': get_description(bs)
        }
        # the ditionary is stored in a dataframe
        df_page = pd.DataFrame(data=spotahome_dict)

        # the temporary data frame gets appended to the data frame we created earlier outside the for loop
        # for every iteration, the data frame page stores the results in the data frame search
        df_search = pd.concat([df_search, df_page], axis=0, ignore_index=True)
    # the data frame search gets returned to the for loop to access it outside the function
    return (df_search)


# creating a list with different property types given from the website
property_types = ['studios', 'apartments/bedrooms:1',
                  'apartments/bedrooms:2', 'apartments/bedrooms:3', 'apartments/bedrooms:3more']
# creating a list with the different location IDs given from the website
locations = [219, 231, 232, 233, 234, 235, 236, 237, 241]

# creating an empty data frame
df_complete = pd.DataFrame()
# for loop to get the different property types
for property_type in property_types:
    # for loop to get the different locations
    for location in locations:
        # append the result from data frame search by calling the function page_results with the property type and the location as a parameter to data frame complete
        df_complete = pd.concat([df_complete, page_results(
            property_type, location)], axis=0, ignore_index=True)


# split the columns 'price' and 'available_from' for further cleaning
df_complete = df_complete.join(df_complete['price'].str.split(
    '-', expand=True).add_prefix('price_'))
df_complete = df_complete.join(df_complete['available_from'].str.split(
    ' ', expand=True).add_prefix('available_from_'))


In [3]:
df_complete


,platform_id,platform,neighbourhood,furniture,property_type,housing_type,price,available_from,available_today,let_type,detailed_furniture,scraping_date,title,price_0,price_1,available_from_0,available_from_1,available_from_2
0,914940,Spotahome,Lambeth,furnished,studios,Studio,1500,21 October,,,furnished,2022-10-21,"Studio for rent in Streatham , London",1500,None,21,October,None
1,913238,Spotahome,Lambeth,furnished,studios,Studio,1400,23 February 2023,,,furnished,2022-10-21,"Studio for rent in Brixton, London",1400,None,23,February,2023
2,113567,Spotahome,Hammersmith and Fulham,furnished,studios,Studio,1400,18 June 2023,,,furnished,2022-10-21,Studio flat with double bed to rent in Kensing...,1400,None,18,June,2023
3,594926,Spotahome,Kensington and Chelsea,furnished,studios,Studio,1800,31 October,,,furnished,2022-10-21,"Studio apartment for rent in Earl's Court, London",1800,None,31,October,None
4,759191,Spotahome,Kensington and Chelsea,furnished,studios,Studio,2300,26 October,,,furnished,2022-10-21,Studio for rent in a co-living building in Not...,2300,None,26,October,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
882,844665,Spotahome,Hackney,furnished,apartments/bedrooms:3more,Apartment,8250-10950,8 January 2023,,,furnished,2022-10-21,4-bedroom apartment for rent in London,8250,10950,8,January,2023
883,843245,Spotahome,Hackney,furnished,apartments/bedrooms:3more,Apartment,9000-11250,11 January 2023,,,furnished,2022-10-21,4-bedroom apartment for rent in London,9000,11250,11,January,2023
884,398966,Spotahome,Hackney,furnished,apartments/bedrooms:3more,Apartment,2100,6 January 2024,,,furnished,2022-10-21,4-bedroom apartment in Hackney,2100,None,6,January,2024
885,602243,Spotahome,Hackney,furnished,apartments/bedrooms:3more,Apartment,9000-11250,4 August 2023,,,furnished,2022-10-21,4 bedrooms warehouse conversion for rent in Ha...,9000,11250,4,August,2023


In [4]:
df_complete.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 887 entries, 0 to 886
Data columns (total 18 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   platform_id         887 non-null    object
 1   platform            887 non-null    object
 2   neighbourhood       887 non-null    object
 3   furniture           887 non-null    object
 4   property_type       887 non-null    object
 5   housing_type        887 non-null    object
 6   price               887 non-null    object
 7   available_from      887 non-null    object
 8   available_today     887 non-null    object
 9   let_type            887 non-null    object
 10  detailed_furniture  887 non-null    object
 11  scraping_date       887 non-null    object
 12  title               887 non-null    object
 13  price_0             887 non-null    object
 14  price_1             253 non-null    object
 15  available_from_0    887 non-null    object
 16  available_from_1    887 no

-----

## Dataframe with information from every detail page for every apartment advert

In [15]:
df_details_complete = pd.DataFrame()

for idx, row in df_complete.iterrows():
    time.sleep(random.randint(2, 6)/10)
    page = requests.get(
        f"https://www.spotahome.com/london/for-rent:{row['housing_type'].lower() + 's'}/{row['platform_id']}")
    html = page.content
    bs = BeautifulSoup(html, 'html.parser')

    details = bs.find(
        'div', class_='property-title__details').find_all('span')

    details_lst = (detail.get_text() for detail in details)
    details_lst = [detail.strip() for detail in details_lst]

    details_lst.pop(0)

    details_lst = [i.split(' ', 1) for i in details_lst]

    row_dict = {}

    for value_key_tuple in details_lst:
        new_key_value = {'id': row['platform_id']}
        row_dict.update(new_key_value)
        key = value_key_tuple[1]
        value = value_key_tuple[0]
        row_dict[key] = value

    df_details = pd.DataFrame(data=row_dict, index=[0])
    df_details_complete = pd.concat([df_details_complete, df_details])


AttributeError: 'NoneType' object has no attribute 'find_all'

In [ ]:
df_details_complete

""
0
0
0
0
0
...
0
0
0
0


In [ ]:
df_details_complete.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 887 entries, 0 to 0
Empty DataFrame

-----

## Import df_complete to DBeaver

In [ ]:
# call the schema created for this project
schema = 'capstone_jmrs'
# get the function to connect to the database
engine = get_engine()

# give the table a unique name
table_name = "spotahome_df_complete"

# import the table to sql
if engine != None:
    try:
        df_complete.to_sql(name=table_name,
                           con=engine,
                           if_exists='replace',
                           schema=schema,
                           index=False,
                           chunksize=5000,
                           method='multi')
        print(f"The {table_name} table was imported successfully.")

    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        engine = None

(psycopg2.OperationalError) could not translate host name "host" to address: nodename nor servname provided, or not known

(Background on this error at: https://sqlalche.me/e/14/e3q8)


## Import df_details_complete to DBeaver

In [ ]:
# call the schema created for this project
schema = 'capstone_jmrs'
# get the function to connect to the database
engine = get_engine()

# give the table a unique name
table_name = "spotahome_df_details_complete"

# import the table to sql
if engine != None:
    try:
        df_details_complete.to_sql(name=table_name,
                                   con=engine,
                                   if_exists='replace',
                                   schema=schema,
                                   index=False,
                                   chunksize=5000,
                                   method='multi')
        print(f"The {table_name} table was imported successfully.")

    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        engine = None


(psycopg2.OperationalError) could not translate host name "host" to address: nodename nor servname provided, or not known

(Background on this error at: https://sqlalche.me/e/14/e3q8)


## Create a .csv file for both data frames

In [ ]:
# create a .csv file with the current date and time
today = dt.datetime.today().strftime('%Y-%m-%d-%H-%M')

df_complete.to_csv('data/spotahome_df_complete_{}.csv'.format(today), sep='\t')
print("The csv-file was created successfully.")

df_details_complete.to_csv('data/spotahome_df_details_complete{}.csv'.format(today), sep='\t')
print("The csv-file was created successfully.")

The csv-file was created successfully.
The csv-file was created successfully.


-----

## Clean the data with SQL

In [ ]:
engine = get_engine()

### drop existing tables because they can't be overwritten

In [ ]:
sql_text = f"""
DROP TABLE IF EXISTS capstone_jmrs.spotahome_merged;
"""
result = engine.execute(sql_text)

OperationalError: (psycopg2.OperationalError) could not translate host name "host" to address: nodename nor servname provided, or not known

(Background on this error at: https://sqlalche.me/e/14/e3q8)

In [ ]:
sql_text = f"""
DROP TABLE IF EXISTS capstone_jmrs.spotahome_eda;
"""
result = engine.execute(sql_text)

In [ ]:
sql_text = f"""
DROP TABLE IF EXISTS capstone_jmrs.spotahome_clean;
"""
result = engine.execute(sql_text)

### join the two tables with the raw data

In [ ]:
sql_text = f"""
CREATE TABLE capstone_jmrs.spotahome_merged AS
SELECT *
FROM capstone_jmrs.spotahome_df_complete sdcaf
LEFT JOIN capstone_jmrs.spotahome_df_details_complete sddc  
	   ON sdcaf.platform_id = sddc.id;
"""
result = engine.execute(sql_text)

### properties who are available this year (2022) don't have the year in it, so we fill it in

In [ ]:
sql_text = f"""
UPDATE capstone_jmrs.spotahome_merged
	SET available_from_2 = COALESCE(available_from_2, '2022');
"""
result = engine.execute(sql_text)

### the date for availability, which was splitted, is merged back into one column

In [ ]:
sql_text = f"""
UPDATE capstone_jmrs.spotahome_merged 
	SET available_from = available_from_0 || '-' || available_from_1 || '-' || available_from_2;
"""
result = engine.execute(sql_text)

## Rename columns

In [ ]:
sql_text = f"""
ALTER TABLE capstone_jmrs.spotahome_merged
	RENAME COLUMN m2 TO size_sqm;
"""
result = engine.execute(sql_text)

### change the column types - dates to DATE, numbers to INT or FLOAT and text to VARCHAR

In [ ]:
sql_text = f"""
ALTER TABLE capstone_jmrs.spotahome_merged
    ALTER COLUMN platform_id TYPE VARCHAR,
    ALTER COLUMN platform TYPE VARCHAR,
    ALTER COLUMN neighbourhood TYPE VARCHAR,
    ALTER COLUMN furniture TYPE VARCHAR,
    ALTER COLUMN housing_type TYPE VARCHAR,
    ALTER COLUMN property_type TYPE VARCHAR,
    ALTER COLUMN size_sqm TYPE FLOAT USING size_sqm::FLOAT,
    ALTER COLUMN bedrooms TYPE FLOAT USING bedrooms::FLOAT,
    ALTER COLUMN bathrooms TYPE FLOAT USING bathrooms::FLOAT,
    ALTER COLUMN price_0 TYPE FLOAT USING price_0::FLOAT,
    ALTER COLUMN price_1 TYPE FLOAT USING price_1::FLOAT,    
    ALTER COLUMN available_from TYPE DATE USING available_from::DATE,
    ALTER COLUMN let_type TYPE VARCHAR,
    ALTER COLUMN detailed_furniture TYPE VARCHAR,
    ALTER COLUMN scraping_date TYPE DATE USING scraping_date::DATE,
    ALTER COLUMN title TYPE VARCHAR;    
"""
result = engine.execute(sql_text)

### drop columns that are no longer needed

In [ ]:
sql_text = f"""   
ALTER TABLE capstone_jmrs.spotahome_merged
  DROP COLUMN available_from_0,
  DROP COLUMN available_from_1,
  DROP COLUMN available_from_2,
  DROP COLUMN id;
  """
result = engine.execute(sql_text)

### square meters who are not logical are replaced with null values (properties with 1m<sup>2</sup> or 1000m<sup>2</sup> are not realistic)

In [ ]:
sql_text = f""" 
UPDATE capstone_jmrs.spotahome_merged
SET size_sqm = NULL
WHERE size_sqm < '10'
	OR size_sqm > '999';
"""
result = engine.execute(sql_text)

### fill missing number of bedrooms with information from property type

In [ ]:
sql_text = f"""
UPDATE capstone_jmrs.spotahome_merged
SET bedrooms = COALESCE(bedrooms, '0')
WHERE property_type = 'studios';
"""
result = engine.execute(sql_text)

In [ ]:
sql_text = f"""
UPDATE capstone_jmrs.spotahome_merged
SET bedrooms = COALESCE(bedrooms, '1')
WHERE property_type = 'apartments/bedrooms:1';
"""
result = engine.execute(sql_text)

In [ ]:
sql_text = f"""
UPDATE capstone_jmrs.spotahome_merged
SET bedrooms = COALESCE(bedrooms, '2')
WHERE property_type = 'apartments/bedrooms:2';
"""
result = engine.execute(sql_text)

In [ ]:
sql_text = f"""
UPDATE capstone_jmrs.spotahome_merged
SET bedrooms = COALESCE(bedrooms, '3')
WHERE property_type = 'apartments/bedrooms:3';
"""
result = engine.execute(sql_text)

In [ ]:
# sql_text = f"""
# UPDATE capstone_jmrs.spotahome_merged
# SET bedrooms = COALESCE(bedrooms, '4')
# WHERE property_type = 'apartments/bedrooms:3more';
# """
# result = engine.execute(sql_text)

### drop columns that are no longer needed

In [ ]:
sql_text = f"""   
ALTER TABLE capstone_jmrs.spotahome_merged
  DROP COLUMN property_type,
	DROP COLUMN title;
"""
result = engine.execute(sql_text)

### rename columns

In [ ]:
sql_text = f"""
ALTER TABLE capstone_jmrs.spotahome_merged
	RENAME COLUMN housing_type TO property_type;
"""
result = engine.execute(sql_text)

### if a property is available 'today' (on the scraping date) set it as 'available', if not, set it as 'occupied'

In [ ]:
today = dt.datetime.today()

In [ ]:
sql_text = f"""
 UPDATE capstone_jmrs.spotahome_merged
	SET available_today = CASE 
      						WHEN available_from = '{today}' THEN 'available'
      						ELSE 'occupied'
						  END;
"""
result = engine.execute(sql_text)

### fill the empty columns with null values

In [ ]:
sql_text = f"""
UPDATE capstone_jmrs.spotahome_merged
SET let_type = NULL 
WHERE let_type = '';
"""
result = engine.execute(sql_text)

## Get the table back to python for the EDA

In [ ]:
spotahome_eda = sf.get_dataframe(f'SELECT * FROM capstone_jmrs.spotahome_merged')
spotahome_eda

-----

## Calculate new columns

### Average price in case there is a price range

In [ ]:
spotahome_eda['price'] = spotahome_eda[['price_0', 'price_1']].mean(axis=1, skipna=True)
spotahome_eda

### Price per square meter

In [ ]:
def my_func(x):
    try:
        return int(x['price']) / int(x['size_sqm'])
    except (ZeroDivisionError, ValueError):
        return np.nan

spotahome_eda["price_sqm"] = spotahome_eda.apply(my_func, axis=1)
spotahome_eda

In [ ]:
spotahome_eda['price_sqm'] = spotahome_eda['price_sqm'].round(decimals = 2)
spotahome_eda

### Price per bedroom

In [ ]:
def my_func(x):
    try:
        return int(x['price']) / int(x['bedrooms'])
    except (ZeroDivisionError, ValueError):
        return int(x['price'])

spotahome_eda["price_bedroom"] = spotahome_eda.apply(my_func, axis=1)
spotahome_eda

In [ ]:
spotahome_eda['price_bedroom'] = spotahome_eda['price_bedroom'].round(decimals = 2)

### Drop column that is no longer needed

In [ ]:
spotahome_eda.drop(columns=['price_0', 'price_1'], inplace=True)

### Some EDA

In [ ]:
spotahome_eda.info()

In [ ]:
spotahome_eda.describe()

In [ ]:
spotahome_eda.corr()

In [ ]:
spotahome_eda.duplicated().sum()

In [ ]:
spotahome_eda.isnull().sum()

In [ ]:
spotahome_eda.dtypes

## Import the table back to DBeaver

In [ ]:
# call the schema created for this project
schema = 'capstone_jmrs'
# get the function to connect to the database
engine = get_engine()

# give the table a unique name
table_name = 'spotahome_eda'

# import the table to sql
if engine != None:
    try:
        spotahome_eda.to_sql(name=table_name,
                                   con=engine,
                                   if_exists='replace',
                                   schema=schema,
                                   index=False,
                                   chunksize=5000,
                                   method='multi')
        print(f"The {table_name} table was imported successfully.")

    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        engine = None


-----

### create the final table with the same order of columns as the table for the other scraped channels

In [ ]:
sql_text = f"""
CREATE TABLE IF NOT EXISTS capstone_jmrs.spotahome_clean AS
SELECT 
	platform_id, 
	platform,
	neighbourhood,
	furniture,
	property_type,
	size_sqm,
	bedrooms,
	bathrooms,
	price,
	price_sqm,
	price_bedroom,
	available_from,
	available_today,
	let_type,
	detailed_furniture,
	scraping_date
FROM capstone_jmrs.spotahome_eda;
"""
result = engine.execute(sql_text)

-----

## Plots

In [ ]:
spotahome_eda['neighbourhood'].value_counts()

In [ ]:
spotahome_eda.plot(figsize=(12,6));

In [ ]:
sns.set(rc = {'figure.figsize':(15,8)})
sns.lineplot(data=spotahome_eda,  palette='Set2', linewidth=2.5, legend='auto');

In [ ]:
sns.set(rc = {'figure.figsize':(15,8)})
sns.lineplot(data=spotahome_eda.price,  palette='Set2', linewidth=2.5, legend='auto', color='red', label='price');

In [ ]:
sns.set(rc = {'figure.figsize':(15,8)})
sns.lineplot(data=spotahome_eda.price_sqm,  palette='Set2', linewidth=2.5, legend='auto', color='red', label='price_sqm');

In [ ]:
sns.set(rc = {'figure.figsize':(15,8)})
sns.lineplot(data=spotahome_eda.price_bedroom,  palette='Set2', linewidth=2.5, legend='auto', color='red', label='price_bedroom');

In [ ]:
sns.pairplot(spotahome_eda);

In [ ]:
spotahome_eda.plot(kind='hist', bins=50, figsize=(12,6), alpha=0.5, density=True, subplots=True, layout=(3,3), sharex=False, sharey=False, legend=True, title='Histograms',  fontsize=10, rot=0);


In [ ]:
spotahome_eda.plot(figsize=(12,6), subplots=True, layout=(3,3), sharex=False, sharey=False, legend=True, fontsize=8, title=' Properties in London', linestyle='solid', linewidth=1, grid=True);


In [ ]:
spotahome_eda.plot(kind='box', figsize=(10, 5), rot=45, fontsize=10, grid=True, title='Boxplot with outliners', color='red', vert=False, patch_artist=True, showfliers=True);


In [ ]:
spotahome_eda.plot(kind='box', figsize=(10, 5), rot=45, fontsize=10, grid=True, title='Boxplot without outliners', color='red', vert=False, patch_artist=True, showfliers=False);


In [ ]:
spotahome_eda.plot(kind='box', figsize=(10, 5), rot=45, fontsize=10, grid=True, title='Boxplot with outliers', color='red', vert=True, patch_artist=True, showfliers=True, subplots=False, sharex=False, sharey=False);


In [ ]:
spotahome_eda.plot(kind='box', figsize=(10, 5), rot=45, fontsize=10, grid=True, title='Boxplot without outliers', color='red', vert=True, patch_artist=True, showfliers=False, subplots=False, sharex=False, sharey=False);


In [ ]:
spotahome_eda.plot(kind='box', figsize=(18, 9), rot=0, fontsize=8, grid=True, title='Boxplot with outliers', color='red', vert=True, patch_artist=True, showfliers=True, subplots=True, layout=(3, 3), legend=True);

In [ ]:
spotahome_eda.plot(kind='box', figsize=(18, 9), rot=0, fontsize=8, grid=True, title='Boxplot without outliers', color='red', vert=True, patch_artist=True, showfliers=False, subplots=True, layout=(3, 3), legend=True);


In [ ]:
spotahome_eda.price.plot(kind='box', figsize=(10, 5), rot=45, fontsize=10, grid=True, title='Price with Fliers', color='red', vert=True, patch_artist=True, showfliers=True, subplots=False, sharex=False, sharey=False);

In [ ]:
spotahome_eda.price.plot(kind='box', figsize=(10, 5), rot=45, fontsize=10, grid=True, title='Price without Fliers', color='red', vert=True, patch_artist=True, showfliers=False, subplots=False, sharex=False, sharey=False);


In [ ]:
spotahome_eda.plot(kind='kde', figsize=(12, 6), fontsize=10, grid=True, title='KDE', color='red', subplots=True, layout=(3, 3), legend=True);


In [ ]:
sns.set(rc = {'figure.figsize':(15,8)})
sns.catplot( kind="box", data=spotahome_eda, height=5, aspect=2, orient='h', palette='Set2');

In [ ]:
sns.heatmap(spotahome_eda.corr(), annot=True, cmap='coolwarm', linewidths=1, linecolor='black');


In [ ]:
spotahome_eda['neighbourhood'].value_counts().plot(kind='bar');

In [ ]:
spotahome_eda.groupby('neighbourhood')['price_sqm'].mean().sort_values().plot(kind='bar');